In [2]:
import pandas as pd
import numpy as np

In [138]:
def buildingFormat(building):
    building = str(building)
    building = building.split("/")[0]
    building = building.split("-")[0]
    return building

def filterInvalidRows(rows):
    rows = rows[pd.notnull(rows['street_number'])]
    rows = rows[rows['street_number'] != "nan"]
    return rows

RULES = [{
        "right": [", просп.", " просп.", ",просп.", ",пр.", " пр."],
        "left_result": "пр. "
    },
    {
        "right": [", пл.", ",пл.", " пл."],
        "left_result": "пл. "
    },
    {
        "right": ["\"гетьм. "],
        "right_result": " гетьм"
    },
    {
        "right": ["\"акад."],
        "right_result": " акад"
    }
]

def streetFormat(street):
    street = str(street)
    for rule in RULES:
        for r in rule["right"]:
            if r in street:
                streetName = street.split(r)[0]
                street = rule["left_result"] + street.split(r)[0] if ("left_result" in rule) else streetName + rule["right_result"]
    return street
def streetFromAddress(street):
    beginings = ['вул.', 'пр.']
    for b in beginings:
        if b in street:
            street = street.split(b)[1]
    street = street.split(',')[0]
    return street
def buildingFromAddress(address):
    return address.split(streetFromAddress(address))[1].split(',')[0]

In [144]:
def cleanFile(file, streetCol, streetNumCol, category, nameCol, sep=","):
    df = pd.read_csv(file, sep=sep)
    result = pd.DataFrame(columns=["street_name", "street_number", "category", "name", "unique_id"])

    result["street_name"] = df[streetCol].map(streetFormat)
    result["street_number"] = df[streetNumCol].map(buildingFormat)
    result["category"] = category
    if (nameCol):
        result["name"] = df[nameCol]
    result = filterInvalidRows(result)
    result["unique_id"] = result.index.map(lambda idx: f'{category} {idx}')
    name = file.split("./")[1].split(".csv")[0] + "__clean" + ".csv"
    result.to_csv(name, encoding='utf-8')

def buildingsFromAddress(file, addressCol, sep=","):
    df = pd.read_csv(file, sep=sep)
    
    df["streetName"] = df['address'].map(streetFromAddress)
    df["buildingNum"] = df['address'].map(buildingFromAddress)
    print(df.head())
    df.to_csv(file, encoding='utf-8')
    
cleanFile('./cafe_lviv.csv', 'street', 'building', 'cafe', 'name')
cleanFile('./museums.csv', 'Street', 'Building_number', 'museum', 'Name')
cleanFile('./hospitals.csv', 'Street', 'Building_number', 'hospital', 'Name', sep=";")
cleanFile('./schools.csv', 'adress_street', 'adress_building', 'school', 'name', sep=";")
cleanFile('./second_hands.csv', 'Street_name', 'Building_number', 'second_hand', 'Name,', sep=";")
# buildingsFromAddress("./universities.csv", "address", sep=";")
cleanFile('./universities.csv', 'streetName', 'buildingNum', 'university', 'Name')

In [142]:
df = pd.read_csv("./universities.csv")
df.head()
df['address']

0                 вул. Університетська,1,м. Львів,79000
1                      вул. С.Бандери,12,м. Львів 79013
2                    вул. Гвардійська,32,м. Львів,79012
3                      вул. Чупринки,105,м. Львів,79057
4                      вул. Пекарська,69,м. Львів,79010
5     вул. В. Великого,1,Жовківський район,м. Дублян...
6                      вул. Пекарська,50,м. Львів,79010
7                     вул. Кубійовича,38,м. Львів,79011
8                      вул. Підвальна,17,м. Львів,79008
9                  вул. Нижанківського,5,м. Львів,79005
10                        пр. Шевченка,9,м. Львів,79005
11                   вул. Менцинського,8,м. Львів,79007
12                  вул. Клепарівська,35,м. Львів,79000
13    смт. Брюховичі,вул. Сухомлинського,16,м. Львів...
14                      вул. Костюшка,11,м. Львів,79000
15                    вул. Городоцька,26,м. Львів,79007
16                  вул. Свєнціцького,17,м. Львів,79011
17                вул. Кульпарківська,99,м. Льві

In [102]:
streetFormat("Абхазька вул.")

'Абхазька вул.'

In [107]:
RULES

[{'right': [', просп.', ' просп.', ',просп.', ',пр.', ' пр.'],
  'result': 'пр. '},
 {'right': [', пл.', ',пл.', ' пл.'], 'result': 'пл. '},
 {'right': [' вул.'], 'result': ''}]